# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [1]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [2]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("df_books.csv", encoding='utf-8')


# Afficher les premières lignes pour vérifier
print(df_books.head())

                                   title   price rating availability
0                   A Light in the Attic  £51.77  Three     In stock
1                     Tipping the Velvet  £53.74    One     In stock
2                             Soumission  £50.10    One     In stock
3                          Sharp Objects  £47.82   Four     In stock
4  Sapiens: A Brief History of Humankind  £54.23   Five     In stock


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [3]:


# Vérification des types de données
# Affiche les 5 premières lignes pour vérifier visuellement les colonnes
print(df_books.head())

# Vérifie les types de colonnes
print("\nTypes de colonnes :")
print(df_books.dtypes)

                                   title   price rating availability
0                   A Light in the Attic  £51.77  Three     In stock
1                     Tipping the Velvet  £53.74    One     In stock
2                             Soumission  £50.10    One     In stock
3                          Sharp Objects  £47.82   Four     In stock
4  Sapiens: A Brief History of Humankind  £54.23   Five     In stock

Types de colonnes :
title           object
price           object
rating          object
availability    object
dtype: object


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [4]:
# # Conversion de 'title' en chaîne de caractères
# df_books["title"] = df_books["title"].astype(str)

# # Vérification du type de la colonne 'title'
# print(df_books["title"].dtype)

Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [5]:
# # Supprimer le symbole '£' et convertir en float
# df_books["price"] = df_books["price"].str.replace("£", "").astype(float)

# # Vérification du type de la colonne 'price'
# print(df_books["price"].dtype)

Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [6]:
# def convert_availability(value):
#     if value == "In stock":
#         return True
#     return False

# df_books["availability"] = df_books["availability"].apply(convert_availability)





Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [7]:
# # Convertir la colonne availability en booléen (True/False)
# # Applique la fonction 'convert_availability' à chaque valeur de la colonne 'availability'
# # Cette fonction transforme le texte ("In stock", "Out of stock", etc.) en booléen : True ou False
# # Exemple : "In stock" → True, "Out of stock" → False
# df_books["availability"] = df_books["availability"].apply(convert_availability)

# # Vérification du type de la colonne availability
# # Affiche le type de données de la colonne 'availability' pour vérifier que la conversion a bien fonctionné
# # Le type attendu est 'bool' (pour booléen), ce qui signifie que chaque valeur est soit True, soit False
# print(df_books["availability"].dtype)

Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [8]:
# # Dictionnaire associant les notes au format texte aux valeurs numériques
# ratings_map = {
#     "One": 1,
#     "Two": 2,
#     "Three": 3,
#     "Four": 4,
#     "Five": 5
# }

# # Utilisation de .map() pour convertir les valeurs textuelles en nombres entiers
# df_books["rating"] = df_books["rating"].map(ratings_map)

# # Vérification du type de la colonne rating
# print(df_books["rating"].dtype)

In [9]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    - 'title' en chaîne de caractères (str)
    - 'price' en float, sans le symbole '£'
    - 'availability' en booléen (True si 'In stock', False sinon)
    - 'rating' en entier (1 à 5)

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """

    # Conversion de 'title' en chaîne de caractères
    df_books["title"] = df_books["title"].astype(str)

    # Nettoyage et conversion de 'price' en float (suppression du symbole £)
    df_books["price"] = df_books["price"].str.replace("£", "").astype(float)

    # Fonction interne pour convertir availability en booléen
    def convert_availability(value):
        if value == "In stock":
            return True
        return False

    df_books["availability"] = df_books["availability"].apply(convert_availability)

    # Dictionnaire de conversion pour rating
    ratings_map = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }

    # Application de la conversion sur la colonne rating
    df_books["rating"] = df_books["rating"].map(ratings_map)

    return df_books


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [10]:
# # Création de la BDD et insertion des données

# import sqlite3

# # Connexion à la base (création si elle n'existe pas)
# connection = sqlite3.connect("book_store.db")

# # Création du curseur pour exécuter les requêtes
# cursor = connection.cursor()

# # 2. Création de la table 'books' si elle n'existe pas déjà
# cursor.execute("""
#     CREATE TABLE IF NOT EXISTS books (
#         id INTEGER PRIMARY KEY AUTOINCREMENT,
#         title TEXT,
#         price REAL,
#         availability BOOLEAN,
#         rating INTEGER
#     )
# """)

# # Connexion à la base SQLite
# conn = sqlite3.connect("book_store.db")

# # Insérer le DataFrame dans la table 'books'
# # if_exists='replace' : remplace la table si elle existe déjà
# # index=False : ne pas insérer l'index du DataFrame comme colonne dans la table
# df_books.to_sql('books', conn, if_exists='replace', index=False)

# # Fermer la connexion
# conn.close()






# 1. Connexion (ou création) de la base de données dans un dossier spécifique
connection = sqlite3.connect("book_store.db")


# On applique les conversions
df_books = convert_types(df_books)

# 2. Insertion du DataFrame dans la base, dans la table 'book_store'
# - 'replace' : remplace la table si elle existe déjà
# - index=False : ne pas insérer l’index du DataFrame comme colonne


df_books.to_sql('book_store', connection, if_exists='replace', index=False)

# 3. Vérification rapide
print("Nombre de modifications :", connection.total_changes)



Nombre de modifications : 1000


Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [11]:
# Compter le nombre de livre dans la BDD
# Connexion à la base de données

cursor = connection.cursor()

# Requête pour compter le nombre de lignes (livres) dans la table
cursor.execute("SELECT COUNT(*) FROM book_store")

# Récupération du résultat
nb_livres = cursor.fetchone()[0]

# Affichage
print("Nombre de livres dans la base :", nb_livres)

# Fermeture
connection.close()

Nombre de livres dans la base : 1000
